In [1]:
# Imports

import pandas as pd
import numpy as np
import re
from Scripts.normalize_original_data import getTrueLabelsDataframe
import Scripts.normalize_original_data as n
import Scripts.functions as fu
from pathlib import Path


In [4]:
df_ground_truth = getTrueLabelsDataframe()